<a href="https://colab.research.google.com/github/Jegovila/SR1/blob/main/Colab/RoboticsToolbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboticstoolbox-python
!pip install ipympl

In [ ]:
# Cambiar el backend para que se muestren los plots
%matplotlib widget

# Imports
import matplotlib.pyplot as plt
import roboticstoolbox as rtb
from roboticstoolbox import RevoluteDH, PrismaticDH, DHRobot, DHLink
from spatialmath import SE3
import numpy as np

# Para desplegar los widgets
from google.colab import output
output.enable_custom_widget_manager()

# Contruir el Robot

**Hay dos formas de definir un robot:**

1. Con **PrismaticDH** o **RevoluteDH**.

Solo introducir los valores que sean diferentes de cero.

In [ ]:
qlim_j1 = [-np.pi/4, np.pi/4]
qlim_j2 = [0, 0.5]
qlim_j3 = [0, 0.5]

bot = DHRobot(
    [
        RevoluteDH(d=0.5, qlim=qlim_j1),
        PrismaticDH(alpha=-np.pi/2, theta=-np.pi/2, offset=0.2, qlim=qlim_j2),
        PrismaticDH(offset=0.2, qlim=qlim_j3)
    ], name="mibot")

2. Con DHLink, en donde habrá que especificar con la variable sigma, el tipo de articulación:

> **1 - Prismática**

> **0 - Revolución**

In [ ]:
bot = DHRobot(
    [
        DHLink(d=0.5, sigma=0),
        DHLink(alpha=-np.pi/2, theta=-np.pi/2, offset=0.2, qlim=[0,0.5], sigma=1),
        DHLink(offset=0.2, qlim=[0,0.5], sigma=1)
    ], name="mibot")

**Nota:** Los límites de las articulaciones "qlim" son necesarias si se queire calcular la cinemática inversa con "ikine" y para graficar el espacio de trabajo.

Imprimir la tabla DH del robot

In [ ]:
bot

También podemos imprimir solo un Link:

In [ ]:
bot[1] # Segundo Link

Graficar robot

In [ ]:
q = [0.1,0.2,0.3]
bot.plot(q)

# Cinemática Directa

In [ ]:
q = [0.1,0.2,0.3]
T = bot.fkine(q)
print("T=")
print(T)

# # Matriz T por columnas
# print(T.n)
# print(T.o)
# print(T.a)
# print(T.t)

También se puede obtener a transformación entre cualquier frame.

```robot.A(j, q)``` Transformación entre frames {0} y {j}.

```robot.A([j1, j2], q)``` Ejemplo entre {j1} y {j2}.

In [ ]:
A1 = bot.A([0,0],q)
A2 = bot.A([1,1],q)
A3 = bot.A([2,2],q)
print(A1 @ A2 @ A3)

## Espacio de trabajo

Las siguientes líneas grafican las posiciones que alcanza el robot. Cada ciclo es para una articulación.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in np.arange(qlim_j1[0], qlim_j1[1], .2):
  for j in np.arange(qlim_j2[0], qlim_j2[1], 0.1):
    for k in np.arange(qlim_j3[0], qlim_j3[1], 0.1):
      T = bot.fkine([i,j,k])
      ax.scatter(T.t[0], T.t[1], T.t[2], s=10, c='red')
plt.show()

# Cinemática inversa

La orientación y la posición deseada deben ser objetos SE3

In [ ]:
Td = SE3.Trans(0.3,0,0.8) * SE3.Rz(-np.pi/2) * SE3.Rx(-np.pi/2)
print(Td)

sol = bot.ikine_LM(Td, joint_limits=True)
print(sol)

Para animar la trayectoria desde una configuración inicial $q_i$ a la solución calculada. La variable "movie" nos guardará un gif que muestra mejor la animación.

In [ ]:
qi = [0, 0, 0]
qt = rtb.jtraj(qi, sol.q, 50)
bot.plot(qt.q, backend='pyplot', movie="mibot.gif")

# Cinemática diferencial

Para calcular el Jacobiano con respecto a la base

In [ ]:
q = [0, 0, 0]
J = bot.jacob0(q)
print(J)

Para el cálculo de las velocidades se requiere el vector de coordenadas generalizadas $q$ y las velocidades articulares $\dot q$

In [ ]:
q = [0, 0, 0]
q_dot = [0.1, 0, 0.3]

J = bot.jacob0(q)
v = J @ q_dot
print(v)

## Cinemática Diferencial Inversa

Cálculo de la cinemática inversa utilizando la cinemática diferencial inversa.

Primero es necesario definir una función que convierta una Transformación SE3 a un vector de movimiento. La transformación va de $T_0$ a $T_1$

In [ ]:
def tr2delta(T0, T1):
    delta = np.array([T1[:3, 3]-T0[:3, 3], 0.5*(np.cross(T0[:3, 0], T1[:3, 0])
                                                + np.cross(T0[:3, 1], T1[:3, 1]) + np.cross(T0[:3, 2], T1[:3, 2]))])
    return delta

$T_0$ es la base, $T_d$ es la transformación final deseada. Se calcula el vector delta deseado $x_d$

In [ ]:
T0 = SE3.Rx(0)
Td = SE3.Trans(0.3,0,0.8) * SE3.Rz(-np.pi/2) * SE3.Rx(-np.pi/2)

T0 = np.array(T0)
Td = np.array(Td)

xd = tr2delta(T0, Td)
xd = xd.reshape((6))
print(xd)

Se calcula ahora el vector delta actual $x$

In [ ]:
qi = [0, 0, 0]

T0 = SE3.Rx(0)
T = bot.fkine(qi)

T0 = np.array(T0)
T = np.array(T)

x = tr2delta(T0, T)
x = x.reshape((6))
print(x)

Cinemática Diferencial Inversa

In [ ]:
iter = 200
q = qi

for i in range(iter):

  # Cálculo de x actual
  T = np.array(bot.fkine(q))
  x = tr2delta(T0, T).reshape((6))

  # Error
  e = xd - x

  J = bot.jacob0(q)

  # Pseudoinversa del Jacobiano (O transpuesta)
  Ji = np.linalg.pinv(J)
  #Ji = np.transpose(J)

  qp = .02 * (Ji @ e)
  q = q + qp

solution = q
print(solution)

Por último, trazar su trayectoria

In [ ]:
qt = rtb.jtraj(qi, solution, 50)
bot.plot(qt.q, backend='pyplot', movie="mibot.gif")